In [1]:
# Please execute this cell first to have the headers ready
# Some websites need to see some browser headers to allow scripts to scrape them.
# Such headers can be found on Google and then be used in requests.get() as we do in this notebook

headers = {
    'pragma': 'no-cache',
    'cache-control': 'no-cache',
    'dnt': '1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
}

In [2]:
#Get the first page to extract page numbers
import requests, re
from bs4 import BeautifulSoup

r=requests.get("http://www.pythonhow.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/", headers=headers) # We use the headers here
c=r.content

soup=BeautifulSoup(c,"html.parser")

all=soup.find_all("div",{"class":"propertyRow"})

all[0].find("h4",{"class":"propPrice"}).text.replace("\n","").replace(" ","")

page_nr=soup.find_all("a",{"class":"Page"})[-1].text
print(page_nr,"number of pages were found")

3 number of pages were found


In [3]:
l=[]
base_url="http://www.pythonhow.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s="
for page in range(0,int(page_nr)*10,10):
    print( )
    r=requests.get(base_url+str(page)+".html", headers=headers)
    c=r.content
    #c=r.json()["list"]
    soup=BeautifulSoup(c,"html.parser")
    all=soup.find_all("div",{"class":"propertyRow"})
    for item in all:
        d={}
        d["Address"]=item.find_all("span",{"class","propAddressCollapse"})[0].text
        try:
            d["Locality"]=item.find_all("span",{"class","propAddressCollapse"})[1].text
        except:
            d["Locality"]=None
        d["Price"]=item.find("h4",{"class","propPrice"}).text.replace("\n","").replace(" ","")
        try:
            d["Beds"]=item.find("span",{"class","infoBed"}).find("b").text
        except:
            d["Beds"]=None
    
        try:
            d["Area"]=item.find("span",{"class","infoSqFt"}).find("b").text
        except:
            d["Area"]=None
    
        try:
            d["Full Baths"]=item.find("span",{"class","infoValueFullBath"}).find("b").text
        except:
            d["Full Baths"]=None

        try:
            d["Half Baths"]=item.find("span",{"class","infoValueHalfBath"}).find("b").text
        except:
            d["Half Baths"]=None
        for column_group in item.find_all("div",{"class":"columnGroup"}):
            for feature_group, feature_name in zip(column_group.find_all("span",{"class":"featureGroup"}),column_group.find_all("span",{"class":"featureName"})):
                if "Lot Size" in feature_group.text:
                    print(feature_name.text)
                    d["Lot Size"]=feature_name.text
        l.append(d)


0.21 Acres
Under 1/2 Acre, 
Under 1/2 Acre, 
0.27 Acres
Under 1/2 Acre, 
Under 1/2 Acre, 
Under 1/2 Acre, 
2.02 Acres


5 Acres
0.7 Acres
3 Acres
Under 1/2 Acre
2.35 Acres
2.05 Acres
0.73 Acres
0.31 Acres


In [4]:
#print(l)

In [5]:
import pandas
df=pandas.DataFrame(l)

In [6]:
df.isnull().sum()

Address        0
Locality       0
Price          0
Beds          19
Area          14
Full Baths    19
Half Baths    36
Lot Size      21
dtype: int64

In [7]:
import math
import numpy as np
import matplotlib.pyplot as plt

In [8]:
new_df = df.copy()

In [9]:
new_df.head()

,Address,Locality,Price,Beds,Area,Full Baths,Half Baths,Lot Size
0,0 Gateway,"Rock Springs, WY 82901","$725,000",None,None,None,None,NaN
1,1003 Winchester Blvd.,"Rock Springs, WY 82901","$452,900",4,None,4,None,0.21 Acres
2,600 Talladega,"Rock Springs, WY 82901","$396,900",5,"3,154",3,None,NaN
3,3239 Spearhead Way,"Rock Springs, WY 82901","$389,900",4,"3,076",3,1,"Under 1/2 Acre,"
4,522 Emerald Street,"Rock Springs, WY 82901","$254,000",3,"1,172",3,None,"Under 1/2 Acre,"


In [10]:
new_df["Beds"].fillna("0", inplace = True)

In [11]:
new_df['Beds'] = new_df['Beds'].apply(int)

In [12]:
new_df['Beds'].replace(0,math.floor(new_df['Beds'].mean()),inplace=True)

In [13]:
new_df["Area"].fillna("0", inplace = True)

In [14]:
new_df['Area'] = new_df['Area'].apply(lambda x:x.replace(",",""))

In [15]:
new_df['Area'] = new_df['Area'].apply(int)

In [16]:
new_df['Area'].mean()

1126.5945945945946

In [17]:
new_df['Area'].replace(0,math.floor(new_df['Area'].mean()),inplace=True)

In [18]:
new_df.head()

,Address,Locality,Price,Beds,Area,Full Baths,Half Baths,Lot Size
0,0 Gateway,"Rock Springs, WY 82901","$725,000",1,1126,None,None,NaN
1,1003 Winchester Blvd.,"Rock Springs, WY 82901","$452,900",4,1126,4,None,0.21 Acres
2,600 Talladega,"Rock Springs, WY 82901","$396,900",5,3154,3,None,NaN
3,3239 Spearhead Way,"Rock Springs, WY 82901","$389,900",4,3076,3,1,"Under 1/2 Acre,"
4,522 Emerald Street,"Rock Springs, WY 82901","$254,000",3,1172,3,None,"Under 1/2 Acre,"


In [19]:
new_df["Full Baths"].fillna("0", inplace = True)

In [20]:
new_df['Full Baths'] = new_df['Full Baths'].apply(int)

In [21]:
new_df['Full Baths'].mean()

1.0810810810810811

In [22]:
new_df['Full Baths'].replace(0,math.floor(new_df['Full Baths'].mean()),inplace=True)

In [23]:
new_df['Price'] = new_df['Price'].apply(lambda x:x.replace(",",""))

In [24]:
new_df['Price'] = new_df['Price'].apply(lambda x:x.replace("$",""))

In [25]:
new_df.head()

,Address,Locality,Price,Beds,Area,Full Baths,Half Baths,Lot Size
0,0 Gateway,"Rock Springs, WY 82901",725000,1,1126,1,None,NaN
1,1003 Winchester Blvd.,"Rock Springs, WY 82901",452900,4,1126,4,None,0.21 Acres
2,600 Talladega,"Rock Springs, WY 82901",396900,5,3154,3,None,NaN
3,3239 Spearhead Way,"Rock Springs, WY 82901",389900,4,3076,3,1,"Under 1/2 Acre,"
4,522 Emerald Street,"Rock Springs, WY 82901",254000,3,1172,3,None,"Under 1/2 Acre,"


In [26]:
new_df.reset_index(drop=True, inplace=True)

In [27]:
new_df.index

RangeIndex(start=0, stop=37, step=1)

In [28]:
X = new_df[['Beds','Area','Full Baths']] #ignoring half baths and lot size columns since it has many null values

In [29]:
y = new_df['Price']

In [30]:
from sklearn.model_selection import train_test_split

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [32]:
from sklearn.linear_model import LinearRegression

In [33]:
lm = LinearRegression()

In [34]:
lm.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [35]:
predictions = lm.predict( X_test)

In [ ]:
plt.scatter(y_test,predictions)
plt.xlabel('Y Test')
plt.ylabel('Predicted Y')

In [ ]:
# calculate these metrics by hand!
from sklearn import metrics

print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [36]:
df.to_csv("Output.csv")

In [ ]:
import requests, re
from bs4 import BeautifulSoup

r=requests.get("http://www.pythonhow.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/", headers=headers)
c=r.content

soup=BeautifulSoup(c,"html.parser")

all=soup.find_all("div",{"class":"propertyRow"})

all[0].find("h4",{"class":"propPrice"}).text.replace("\n","").replace(" ","")

page_nr=soup.find_all("a",{"class":"Page"})[-1].text
print(page_nr,"number of pages were found")

base_url="http://www.pythonhow.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s="
for page in range(0,int(page_nr)*10,10):
    print( )
    r=requests.get(base_url+str(page)+".html")
    c=r.content
    #c=r.json()["list"]
    soup=BeautifulSoup(c,"html.parser")
    all=soup.find_all("div",{"class":"propertyRow"})
    
#print(all)